In [1]:
import pandas as pd
df_books = pd.read_csv('Combined books.csv - Overall Form.csv')

From kumu.io 's documentation.
The `elements.csv` and `connections.csv` must be in this form.

```
#To be included in the elements form
#Label
#Type (optional)
#Description (optional)

#For connections
#
#To be included in the connections form
#From
#To
#Type (optional)
#Label (optional)
#
#Here's a sample of how your element spreadsheet might look:

# Label   Type    Description Tags
# Jeff Mohr   Person  Co-Founder and CEO, always up for learning Founder|Male|Runner
```

In [2]:
import re
AUTHOR_NAMES_SEP_PATTERN = re.compile(r',|&')
AUTHOR_NAMES_NONEXIST_PATTERN = re.compile(r'^\s*$')
TITLE_TERMS_SEP_PATTERN = re.compile(r'[\s,:]')

elements = []
connections = []

for (i, row) in df_books.iterrows():
    category = row['Category']
    row['Author'] = row['Author'].replace('\xc2\xa0', ' ').replace('\xe2\x80\x99', "'")
    authors = AUTHOR_NAMES_SEP_PATTERN.split(row['Author'].strip().strip('\xc2\xa0'))
    authors = filter(lambda x: not AUTHOR_NAMES_NONEXIST_PATTERN.match(x), authors)
    authors = map(str.strip, authors)
    for author in authors:
        elements.append((author, 'Author', ''))
    #print authors
    title = row['Title']
    title_terms = TITLE_TERMS_SEP_PATTERN.split(title)
    title_terms = filter(lambda x: x not in [
            'and', 'or', 'in', 'a', 'A', 'an', 'An', 'And', 'Or', 'In', 
            'of', 'Of', 'to', 'into', 'Into', 'the', 'The', '',
            'with'
        ], title_terms)
    for term in title_terms:
        elements.append((term, 'Term', ''))
    elements.append((category, 'Category', ''))
    year = row['Year']
    elements.append((year, 'Year', ''))
    elements.append((title, 'Title', ''))
    
    connections.append((category, title, '', ''))
    connections.append((title, year, '', ''))
    for author in authors:
        connections.append((author, title, '', ''))
    for term in title_terms:
        connections.append((title, term, '', ''))

    
    #print title_terms

In [3]:
pd.DataFrame(elements, columns=['Label', 'Type', 'Description']).drop_duplicates().to_csv('elements.csv', index=False)
pd.DataFrame(connections, columns=['From', 'To', 'Type', 'Label']).to_csv('connections.csv', index=False)